# Agnostische EDA auf die Zielvariable
## Eine gezielte Untersuchung der Daten auf mögliche Prädiktoren ohne die Scheuklappen des Vorwissens

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    <b>Das Szenario</b><br>
    In dieses Projekt schauen wir ohne Domänenwissen über die Daten: Wir sollen einen Datensatz als Vorbereitung
    für ein späteres Modelltraining so unvoreingenommen wie möglich auf gute Kandidaten zur Vorhersage einer
    gegebenen Zielvariable untersuchen. Dazu wurden sowohl die Spalten des Originaldatensatzes umbenannt und 
    so in einen völlig anderen, fiktiven Kontext gesetzt, als auch insbesondere Kategorienklassen diesem 
    fiktiven Kontext angepasst.<br>
    Unser Wissen über die Daten beschränkt sich auf das Folgende:<br>
    <li>Die Daten liegen in einer .csv-Datei namens "eda_data.csv"</li>
    <li>Die erste Spalte "ID" ist eine numerische, lückenlos aufsteigende Indexspalte ohne Dopplungen</li>
    <li>Die Zielvariable für das spätere Modelltraining heißt "Infected"</li>
    <br>
    <b>Das Ziel dieser EDA</b><br>
    In diesem Notebook gehen wir exemplarisch durch, wie wir in der EDA
    <li>den Blick für mögliche wichtige Spalten offen halten, auch und gerade wenn unser Domänenwissen 
    und/oder "der gesunde Menschenverstand" bestimmte Features als wichtig nahelegen und andere verfrüht 
    als vernachlässigbar erscheinen lassen, und</li>
    <li>eine vielversprechende Featureselektion und ggf. sogar ein vielversprechendes erstes Feature Engineering
    für ein einfaches Basismodell vorbereiten können, das dennoch bereits gute Ergebnisse liefert.</li><br>
    <br>
    <b>Die Daten</b><br>
    Tatsächlich handelt es sich hier um einen Auszug aus einem authentischen Datensatz rund um die Passgiere 
    der HMS Titanic, der aber wie oben beschrieben vorverarbeitet und durch einige fiktive Daten ergänzt wurde. 
    Findest Du heraus, welche Spalten ursprünglich welchen Inhalt hatten, ohne in die Auflösung zu schauen? 
    (ganz unten im grünen Kasten)
</div>

## Vorbereitung

In [ ]:
# Importiere alle benötigten Module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Lege Darstellungsoptionen, Warnungsunterdrückungen etc. fest
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
# Lade die Daten
df = pd.read_csv('eda_data.csv')

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    <b>In der EDA nehmen wir diverse Datenmanipulationen vor.</b><br>
    Um sicherzugehen, dass wir hier nichts an den Daten ändern, die wir später im Training verwenden, arbeiten wir daher in der EDA <b>ausschließlich mit einer Kopie der Daten.</b><br>
    Eine Alternative wäre, die Daten nach der EDA neu einzulesen - gerade bei großen Datensätzen wird dadurch weniger lokaler Speicherplatz belegt, aber dafür u.a. die Ladezeiten erhöht.
</div>

In [ ]:
# Erstelle eine Kopie der Rohdaten für die EDA
df_eda = df.copy()

#### Ein erster Blick auf die Daten

In [ ]:
# Betrachte die Daten
df_eda.head()

In [ ]:
# Zeige eine Übersicht
def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))
overview(df_eda)

In [ ]:
Nan-Werte
Nans bei Vienna für o3 checken
Buenos Aires

[Link](https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions) zur Dokumentation zu list comprehensions

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    <b>Machen wir uns ein paar Notizen.</b><br>
</div>

<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    <b>Zielvariable</b><br>
        #Infected: cat, binär -> binäres Klassifikationsproblem.<br>
    <b>Weitere Variablen:</b><br>
        <li>#Setup: cat, 3 Klassen (ordinal?)</li>
        <li>#Type: cat, binär</li>
        <li>#Level: float16, 20% missing</li>
        <li>#Local_Pool: Int8, 9 unique. (kategorisch?)</li>
        <li>#M_Label: cat, 3 Klassen, <1% nans</li>
        <li>#Building: cat, 8 Klassen (ordinal?), 77% missing</li>
        <li>#Licence: cat, 76% uniques (reduzierbar?)</li>
        <li>#M_Number: Int16</li>
        <li>#P_Number: float16, 48% missing</li>
        <li>#Last_User: cat, 100% unique</li>
</div>

In [ ]:
# Nahbetrachtungen
print(f'Duplikate: {df_eda.duplicated().sum()}')
print(f'Uniques in "Version": {df_eda["Level"].unique()}')
display(df_eda.describe())

# Gruppierung
cats = ['Setup', 'Type', 'Local_Pool', 'M_Label', 'Building']
print(cats)
nums = list(df_eda.select_dtypes(exclude="object").columns)
print(nums)

[Link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html) zur Dokumentation von `select_dtypes()`


Eine andere Möglichkeit für Notizen:

In [ ]:
''' NOTIZEN

#Infected (Target): cat, binär -> binäres Klassifikationsproblem

#Setup: cat, 3 Klassen (ordinal?)
#Type: cat, binär
#Level: float16, 177 nan
#Local_Pool: Int8, 9 unique (kategorisch?)
#M_Label: cat, 3 Klassen, 2 nan
#Building: cat, 8 Klassen (ordinal?), 687 nan -> nur 204 given (!)
#Licence: cat, 681 uniques (reduzierbar?)
#M_Number: Int16
#P_Number: float16
#Last_User: cat, alle unique (reduzierbar?)
'''

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    <b>Kategorische Features mit sehr vielen einzigartigen Werten müssen oft erst sehr aufwändig 
    analysiert und aufbereitet werden,</b> um ihnen für das Modelltraining wertvolle Informationen 
    zu entlocken, ohne die Dimensionalität bis zur Unbrauchbarkeit zu expandieren - <b>falls sie 
    überhaupt wertvolle Informationen für das Modelltraining enthalten.</b> In diesem Beispiel 
    bietet die Spalte 'Licence' viel Raum zum Experimentieren. Man könnte sie zum Beispiel auf 
    Fragen untersuchen wie "sind die Zahlenteile der Werte als numerische Features ein wertvoller 
    Prädiktor? Oder umgekehrt die alphanumerischen Anteile?" etc.pp. Im Rahmen sowohl der Live 
    Session als auch einer EDA würde dies aber zu weit führen, und auch in der Praxis lautet 
    unsere Empfehlung im Umgang mit dieser Art von Features:<br>
    <b>Solange es keinen zwingenden Grund gibt, diese Features bereits im Basismodell zu 
    berücksichtigen, lasse sie zumindest zunächst außen vor und komme erst in späteren 
    Iterationen der Modelloptimierung ggf. gezielt auf sie zurück. 
    Die mögliche Verbesserung der Modellperformance und der dafür notwendige Aufwand stehen hier 
    oft in einem eher fragwürdigen Verhältnis zueinander.</b>
</div>

#### Vorbereitung der Daten

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    Die Zielvariable ist <b>binär kategorisch</b>,<br>
    und um sie möglichst flexibel auswerten zu können, kodieren wir sie ensprechend.
</div>

In [ ]:
# Zielvariable binär kodieren
df_eda['Infected'] = df_eda['Infected'].replace({'no': 0, 'yes': 1})
df_eda.head()

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    <b>Nicht selten ist der bloße Umstand an sich, dass Werte fehlen, eine wertvolle Information</b> 
    auch und gerade für die Vorhersage der Zielvariablen. Daher lohnt es sich oft, in der EDA zunächst 
    "auf Verdacht" davon auszugehen, dass dem so ist, und machen diese Information auswertbar:
    <li>In kategorischen Spalten ersetzen wir fehlende Werte durch einen eigenen einzigartigen <b>Platzhalter-Wert</b>, 
    z.B. "Unknown", "missing", "keine_Angabe" o.ä.: <code>df['Feature]' = df['Feature'].fillna("Unknown")</li>
    <li>In numerischen Spalten können wir fehlende Werte nicht einfach durch einen Platzhalter wie 0 ersetzen, 
    da dies einerseits die Werteverteilung verzerren würde und andererseits die Information, dass 
    der Wert fehlte, verloren ginge. Stattdessen erstellen wir ein neues binäres, sogenanntes 
    <b>Missingness</b>-Feature: <code>df['Feature_missing'] = df['Feature'].isna()</li>
</div>

In [ ]:
# Nochmal eine Übersicht zu fehlenden Werten ziehen
df_eda.isna().sum()

<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    Interessante Spalten: Level, Building, P_Number
</div>

In [ ]:
df_eda['P_Number'].isna().values.any()

In [ ]:
# Für numerische Features mit signifikant vielen fehlenden Werten: "Missingness"-Feature
nums = list(df_eda.select_dtypes(exclude = 'object').columns)
print(nums)

for entry in nums:
    if df_eda[entry].isna().values.any():
        name = entry + '_missing'
        df_eda[name] = df_eda[entry].isna().astype('uint8')

df_eda.head()

In [ ]:
df_eda.columns

In [ ]:
# Nicht vergessen: Neue kategorische Spalten in die cats-Liste aufnehmen
cats = list(df_eda.select_dtypes(include=['object']).columns)
cats = cats + ['Level_missing', 'P_Number_missing']
cats

# Bei kategorischen Features: Eigene Klasse 'Unknown'
for entry in cats:
    df_eda[entry] = df_eda[entry].fillna('Unknown')

In [ ]:
# Noch einmal den Überblick, um zu prüfen, ob alles geklappt hat
overview(df_eda)

## Und jetzt: Plotten, plotten und plotten
und dabei Notizen machen

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    Beim Plotten von Verteilungen, Strukturen etc. bringt es einen erheblichen Mehrwert für die 
    Auswertung zum Zweck eines besseren Modelltrainings, wenn wir insbesondere die ggf. 
    unterschiedlichen <b>Verteilungen der Zielklassen in den Features darstellen.</b><br>
    Dafür gibt es verschiedene Möglichkeiten:
    <li> Mit Kreuztabellen (<code>pd.crosstab(index=df['Feature'], columns=df['Target'])</code>)</li>
    <li> Mittels Gruppierung (<code>df.groupby('Target')['Feature']</code>)</li>
    <li> In vielen Seaborn-Plots mit dem Parameter <code>hue='Target'</code></li>
</div>

#### Zuerst: Die Zielvariable

In [ ]:
# Was ist das Verhältnis zwischen den Zielklassen ganz allgemein?
pd.crosstab(index = df['Infected'], columns = 'count', normalize = True)



<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    Die Zielklassen sind mit einem Verhältnis von 3:2 nicht ganz ausbalanciert, aber für das Basismodell 
    wahrscheinlich ausreichend. Resampling ist also voraussichtlich erst in der Phase der iterativen 
    Modelloptimierung notwendig.
</div>

#### Beginnen wir mit den numerischen Features

In [ ]:
# Korrelogramm aller numerischen Spalten inklusive den numerisch kodierten Kategorien
fig, ax = plt.subplots(figsize = [10,8])
sns.heatmap(data = df_eda.corr(), annot = True, ax = ax)


<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    <b>Korrelationen ~ Infected</b>
    <li>M_Number: 0,6</li>
    <li>P_Number: 0,41</li>
    <li>P_Number_missings: 0,39</li>
    <b>Korrelationen untereinander</b>
    <li>M_Number ~ P_Number: 0,86</li>
    <li>Level ~ P_Number: 0,4</li>
    <li>Level ~ M_Number: 0,38</li>
    <li>Level ~ Local_Pool: -0,3</li>
    <li>Local_Pool ~ P_Number: -0,21</li>
</div>

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    In erster Linie sind für uns natürlich die unmittelbaren Korrelationen mit der Zielvariablen 
    von Bedeutung. Weitere Korrelationen können uns aber gute Hinweise für die Feature Selection 
    und das Feature Engineering geben, z.B. (als Orientierungsleitlinie zu verstehen, nicht als 
    feste Werte):
    <li>Absolute Korrelation = 1: Die Spalten sind redundant, und es sollte nur eine von ihnen 
    für das Modelltraining verwendet werden.
    <li>0,8 < Absolute Korrelation < 1: Kolinearität. Der Einfluss auf die Zielvariable ist 
    bestenfalls schwer den einzelnen Spalten zuzuordnen; Zusammenfassungsmethoden wie PCA erwägen</li>
    <li>usw.</li>
</div>

In [ ]:
# Baue eine Funktion zum Plotten der numerischen Features
def numplots(col, data=df_eda, key='Infected'):
    '''Plot a grouped histogram and boxplot
    ARGS
        col: Column to plot
        data: DataFrame (default: eda)
        key: Column to group by (default: 'Infected')
    RETURN: Nne
    '''
    fig, ax = plt.subplots(ncols=3, figsize=(16,3))
    sns.boxplot(data=data, y=col, x=key, ax=ax[0])
    data.groupby(key)[col].plot(kind='hist', bins=20, ax=ax[1], alpha=0.5)
    data.groupby(key)[col].plot(kind='kde', ax=ax[2])

In [ ]:
# Vorbereitung
nums

nums = [
 'Level',
 'Local_Pool',
 'M_Number',
 'P_Number',
 'Level_missing',
 'P_Number_missing']

In [ ]:
# Erstelle die Plots für die numerischen Features
for i in nums:
    numplots(i, df_eda)


<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    <li>M-Number und P_Number scheinen wichtig</li>
    <li>Level und Local_Pool eher weniger</li>
    <li>P_Number: Ausreißer?</li>
</div>

In [ ]:
# Scattermatrix / Pairplot gruppiert nach Zielvariable
sns.pairplot(data = df_eda, hue = 'Infected')

<div style="border: 1px solid black; padding: 10px; background-color: lightblue;">
    <b>Tipp:</b> Mit einem Alpha-Kanal (über den Parameter <code>plot_kws={'alpha': 0.2}</code>) wirkt der Plot unscharf, aber es kann dazu beitragen, Muster visuell intuitiver zu erkennen.
</div>

<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    <li>Ausreißer-Erkennung nach Level ~ M_Number? (DBSCAN/RANSAC/KNN?)</li>
    <li>Polynomials über P_Number vor der PCA mit M_Number?</li>
    <li> ...</li>
</div>

#### Und nun zu den Kategorien

In [ ]:
# Bauen wir uns eine Funktion zum Plotten der Kategorien
def catplot(x, y=df_eda['Infected']):
    '''Display a barplot of a normalized crosstab between x and y and an absolute crosstab for a sanity check.
    ARGS:
        x: Crosstab 'index' column
        y: Crosstab 'columns' column (default: eda['Infected'])
    RETURNS: None
    '''
    # Create two crosstabs: One for absolute and one for relative distributions
    crosstab_abs = pd.crosstab(index=df_eda[x], columns=y)
    crosstab_rel = pd.crosstab(index=df_eda[x], columns=y, normalize='index')
    
    # Plot them side by side
    fig, ax = plt.subplots(figsize=(16,3), ncols=2)
    crosstab_abs.plot(kind='bar', ax=ax[0])
    ax[0].set_title('Absolut')
    crosstab_rel.plot(kind='bar', ax=ax[1])
    ax[1].set_title('Normalisiert')

In [ ]:
# Plotte die Kategorien in cats
for entry in cats:
    catplot(entry)


<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    <li>Setup: Anscheinend ordinal -> ggf. ordinal kodieren* oder numerisch transformieren statt OHE</li>
    <li>Type: Wichtiger Prädiktor</li>
    <li>Local_Pool: Binäres Feature "LocalPoolIn123" testen</li>
    <li>M_Label: Evtl. ordinale Kodierung testen</li>
    <li>Building: Binäres Feature "BuildingKnown" testen</li>
    <li>Level_missing: Unterschied erkennbar, aber vergleichsweise schwach</li>
    <li>P_Number_missing: Wichtiger Prädiktor</li>
</div>

*[Link](https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) zur Dokumentation zum `OrdinalEncoder()`

In [ ]:
df_eda['Setup'].unique()

In [ ]:
# Engineered Features
df_eda['Setup'] = df_eda['Setup'].replace({'3rd': 3, '2nd': 2, '1st': 1})

In [ ]:
# neues numerisches Feature -> neue Korrelationsmatrix
fig, ax = plt.subplots(figsize = [10,8])
sns.heatmap(data = df_eda.corr(), annot = True, ax = ax)

#### Zusammenfassung

<div style="border: 1px solid black; padding: 10px; background-color: lightyellow;">
    <b>Features für das Basismodell</b>
        <li>#Type (Im Cleaning in boolean oder [0,1] transformieren)</li>
        <li>#Setup</li>
        <li>#M_Number</li>
        <li>#P_Number</li>
        <li>#P_Number_missing</li>
        <li>#LocalPoolIn123</li>
        <li>#BuildingKnown</li>
    -> Nur numerische oder binäre Features im Basismodell<br>
    <b>In iterativer Modelloptimierung testen</b>
        <li>Resampling</li>
        <li>#M_Label (ordinal oder OHE)</li>
        <li>#Level_Missing</li>
        <li>PCA (#M_Number, #P_Number)</li>
        <li>Polynomial (#P_Number)</li>
        <li>Ausreißererkennung (#Level ~ #M_Number)</li>
        <li>#Licence aufbohren</li>
</div>

<div style="border: 1px solid black; padding: 10px; background-color: lightgreen;">
    <b>Auflösung:</b>
    <li>'Infected' ['yes','no']: 'Survived'; ob ein Passagier den Untergang der Titanic überlebt hat oder nicht</li>
    <li>'Setup' ['1st','2nd','3rd']: 'Pclass'; die Kabinenklasse</li>
    <li>'Type' ['PC','Mac']: 'Sex' ['m','f']; das Geschlecht des Passagiers</li>
    <li>'Level': 'Age'; Alter des Passagiers</li>
    <li>'Local_Pool': 'Family'; Wie viele Familienangehörige mit an Bord waren</li>
    <li>'M_Label' ['S','C','Q']: 'Embarked'; Einstiegshafen (Southhampton, Cherbourg oder Queenstown)</li>
    <li>'Building' ['A','B','C','D','E','F','G','T']: 'Deck'; Kabinendeck</li>
    <li>'Licence': 'Ticket'; Registriernummer des Tickets</li>
    <li>'M_Number' & 'P_Number': Künstlich erzeugte, fiktive Variablen ohne inhaltliche Bedeutung</li>
    <li>'Last_User': 'Name'; Name des Passagiers</li>
</div>